# Bitcoin Price Prediction Project

Data source - Coinpaprika API Python Client:
- Web Site - https://coinpaprika.com/waluta/btc-bitcoin/
- Github   - https://github.com/s0h3ck/coinpaprika-api-python-client

### Requirements in requirements.txt file, for example:
coinpaprika==0.1.0

In [1]:
# pip install coinpaprika

In [2]:
from coinpaprika import client as Coinpaprika

client = Coinpaprika.Client()

# List coins
# client.coins()

# Get coin by ID (example: btc-bitcoin)
# client.coin("btc-bitcoin")

# Get tweets by coin ID (max 50 tweets)
# client.twitter("btc-bitcoin")

In [3]:
# Get historical OHLCV information for a specific coin (USD,BTC)

client.candles("btc-bitcoin", quotes="USD", start="2014-01-11T00:00:00Z")

[{'time_open': '2014-01-11T00:00:00Z',
  'time_close': '2014-01-11T23:59:59Z',
  'open': 867.32,
  'high': 921.48,
  'low': 861.72,
  'close': 913.95,
  'volume': 44754200,
  'market_cap': 11195636163}]

In [4]:
# Get historical information for a specific coin (USD,BTC)
# time period, default time delta: 5min
# end="2022-02-09T08:05:00Z"  ==>  end at ... 08:00:00

client.historical("btc-bitcoin", quotes="USD", start="2022-02-06T00:00:00Z",  end="2022-02-09T08:05:00Z")

[{'timestamp': '2022-02-06T00:00:00Z',
  'price': 41497.46,
  'volume_24h': 21861799679,
  'market_cap': 786366573815},
 {'timestamp': '2022-02-06T00:05:00Z',
  'price': 41498.77,
  'volume_24h': 21690386761,
  'market_cap': 786391303477},
 {'timestamp': '2022-02-06T00:10:00Z',
  'price': 41498.79,
  'volume_24h': 22966940474,
  'market_cap': 786391725926},
 {'timestamp': '2022-02-06T00:15:00Z',
  'price': 41503.06,
  'volume_24h': 22864785576,
  'market_cap': 786472697527},
 {'timestamp': '2022-02-06T00:20:00Z',
  'price': 41518.61,
  'volume_24h': 22737621377,
  'market_cap': 786768125925},
 {'timestamp': '2022-02-06T00:25:00Z',
  'price': 41547.85,
  'volume_24h': 22557041775,
  'market_cap': 787322132444},
 {'timestamp': '2022-02-06T00:30:00Z',
  'price': 41517.84,
  'volume_24h': 22580774434,
  'market_cap': 786753549589},
 {'timestamp': '2022-02-06T00:35:00Z',
  'price': 41461,
  'volume_24h': 22433949706,
  'market_cap': 785677242947},
 {'timestamp': '2022-02-06T00:40:00Z',
  'p